# Continuação do curso de otimização de hiperparâmetros 1

## Dados

In [31]:
import pandas as pd
import numpy as np

In [32]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [33]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


## Explorando aleatoriamente com o RandomizedSearchCV

Já aprendemos que, quando temos um espaço de parâmetros com duas dimensões, podemos explorá-lo ponto a ponto. Isto é, transformamos espaços contínuos em espaços discretos e exploramos, nesses pontos, o nosso algorítimo.

Por exemplo, se estamos trabalhando com um algorítimo de DecisionTreeClassifier que tem os parâmetros max_depth e min_samples_leaf, podemos testar cada um desses parâmetros com um valor específico. Depois de medirmos o resultado, repetimos o processo para o próximo parâmetro.

Dessa forma, exploramos o espaço até completarmos o grid todo. Por exemplo, se temos 15 condições para cada parâmetro, rodamos o algorítimo 225 vezes para explorar esse espaço por completo.

Como a exploração aumenta exponencialmente a quantidade de vezes que rodamos o algoritmo, ao invés de tentarmos explorar todo o grid (o que é feito no grid search), poderíamos buscar pontos aleatoriamente (random search). E é exatamente isso que faremos agora.

Para organizarmos nosso trabalho, adicionaremos uma célula de texto indicando onde se inicia o RandomSearch. Esse processo de busca é bastante parecido com tudo o que fizemos anteriormente, e também se inicia definindo um espaço de parâmetros a ser explorado.

Portanto, começaremos copiando o código que criamos para GridSearchCV:

In [34]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier

SEED=301
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth" : [3, 5],
    "min_samples_split": [32, 64, 128],
    "min_samples_leaf": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

busca = GridSearchCV(DecisionTreeClassifier(),
                     espaco_de_parametros,
                     cv = KFold(n_splits = 5, shuffle=True))

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009923,0.000755,0.001711,0.000070,gini,3,32,32,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.784,0.776,0.8025,0.793,0.7795,0.787,0.009618,1
1,0.009447,0.000079,0.001649,0.000011,gini,3,32,64,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.784,0.776,0.8025,0.793,0.7795,0.787,0.009618,1
2,0.009474,0.000060,0.001657,0.000013,gini,3,32,128,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.784,0.776,0.8025,0.793,0.7795,0.787,0.009618,1
3,0.010419,0.001475,0.002038,0.000450,gini,3,64,32,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.784,0.776,0.8025,0.793,0.7795,0.787,0.009618,1
4,0.010136,0.001270,0.001706,0.000060,gini,3,64,64,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.784,0.776,0.8025,0.793,0.7795,0.787,0.009618,1


Em seguida, alteraremos os campos em que GridSearchCV aparece para RandomizedSearchCV. Manteremos a mesma SEED e o mesmo espaço de parâmetros (com 36 possibilidades).

Dentre essas 36 possibilidades de combinações de parâmetros, quantas queremos rodar? Se executarmos todas, estaremos fazendo exatamente a mesma busca que com o GridSearchCV, alterando apenas a ordem. Ou seja, devemos executar somente algumas.

Um dos parâmetros que RandomizedSearchCV pode receber é o número de iterações - n_iter. A ideia é, nesse momento, rodarmos apenas 16 dessas possibilidades:

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

SEED=301
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth" : [3, 5],
    "min_samples_split": [32, 64, 128],
    "min_samples_leaf": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                           espaco_de_parametros,
                           n_iter = 16,  #aqui
                           cv = KFold(n_splits = 5),
                           random_state = SEED)


busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012768,0.000362,0.001707,0.000051,128,128,5,gini,"{'min_samples_split': 128, 'min_samples_leaf':...",0.4330,0.4525,0.771,0.742,0.7725,0.6342,0.156817,1
1,0.009948,0.001018,0.001683,0.000042,64,32,3,gini,"{'min_samples_split': 64, 'min_samples_leaf': ...",0.4015,0.4165,0.771,0.766,0.7725,0.6255,0.176848,11
2,0.009396,0.000172,0.001714,0.000061,64,128,3,gini,"{'min_samples_split': 64, 'min_samples_leaf': ...",0.4015,0.4165,0.771,0.766,0.7725,0.6255,0.176848,11
3,0.018793,0.000868,0.001746,0.000034,32,64,5,entropy,"{'min_samples_split': 32, 'min_samples_leaf': ...",0.4240,0.4535,0.771,0.750,0.7445,0.6286,0.155544,5
4,0.018705,0.000773,0.001779,0.000093,64,64,5,entropy,"{'min_samples_split': 64, 'min_samples_leaf': ...",0.4240,0.4535,0.771,0.750,0.7445,0.6286,0.155544,5


Após a execução desse código, queremos saber quão bem se saiu o melhor classificador. Da mesma forma que no GridSearchCV, encontraremos uma resposta com cross_val_score() (nested cross validation).

In [36]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))
scores

array([0.7755, 0.78  , 0.8055, 0.7855, 0.774 ])

O resultado na tela será um array de cinco valores.

Trazendo para este notebook a função de imprimir scores:

In [37]:
def imprime_scores(scores):
  media = scores.mean() * 100
  desvio = scores.std() * 100
  

  print(f'Accuracy médio: {media:.2f}%')
  print(f'Intervalo: {media - 2 * desvio:.2f}% a {media + 2 * desvio:.2f}%')

Temos:

In [38]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits=5, shuffle=True))

imprime_scores(scores)

Accuracy médio: 78.69%
Intervalo: 76.70% a 80.68%


Em seguida, para encontrarmos o melhor estimador, atribuiremos a função busca.best_estimator_ à uma variável melhor e imprimiremos essa variável na tela.

In [39]:
melhor = busca.best_estimator_
melhor

DecisionTreeClassifier(max_depth=5, min_samples_leaf=128, min_samples_split=128)

Isso significa que o melhor estimador teve a profundidade máxima 5, o mínimo de elementos na folha 128 e, e 128 como o mínimo de splits antes de tomar uma decisão. Tivemos uma acurácia média de 78.69%, em um intervalo entre 76.70% e 80.68%.

Repare que executando menos da metade das buscas, obtivemos uma acurácia média e um intervalo muito parecidos com aqueles do GridSearchCV (que tinha a média 78.68% e o intervalo 76.85% a 80.55).

Na prática, a utilização do RandomizedSearchCV nos permite encontrar valores muito próximos aos que mais otimizarão nossos estimadores, sem que seja necessário explorar todo o espaço de parâmetros (o que muitas vezes é impossível).

# Customizando o espaço de hiper parâmetros

Nós exploramos aleatoriamente o nosso espaço de parâmetros, mas fizemos isso de maneira bem restrita. Anteriormente, devido às limitações de processamento do GridSearchCV (principalmente em relação ao tempo), nós utilizamos somente 36 combinações.

Porém, seria mais interessante explorarmos ainda mais parâmetros no nosso algorítimo - por exemplo, um max_depth que recebesse 10, 20, 30 ou até que não tivesse limites (o que é possível com None).

A ideia é executarmos novamente o RandomizedSearchCV, mas com diferentes customizações nesse espaço de parâmetros. Por exemplo, em max_depth, ao invés de termos somente os valores 3 e 5, teremos um conjunto discreto de números inteiros (3, 5, 10, 15, 20, 30) com a adição do valor None.

Isso significa que agora temos muito mais possibilidades de combinações: são 7 elementos para max_depth, 96 para min_samples_split e min_samples_leaf, e 2 para criterion - no total, 129.024 combinações diferentes de parâmetros.

Desse número, executaremos apenas 16, a mesma quantidade que estávamos executando anteriormente, mas com um espaço de parâmetros muito maior e mais complexo:

In [40]:
from scipy.stats import randint

SEED=301
np.random.seed(SEED)

espaco_de_parametros = {
    "max_depth" : [3, 5, 10, 15, 20, 30, None],
    "min_samples_split" : randint(32, 128),
    "min_samples_leaf" : randint(32, 128),
    "criterion" : ["gini", "entropy"]
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                           espaco_de_parametros,
                           n_iter = 16,
                           cv = KFold(n_splits = 5, shuffle=True),
                           random_state = SEED)

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.016346,0.006150,0.001790,0.000102,entropy,3,71,100,"{'criterion': 'entropy', 'max_depth': 3, 'min_...",0.7840,0.7760,0.8025,0.7930,0.7795,0.7870,0.009618,1
1,0.017372,0.000186,0.001836,0.000037,gini,15,93,111,"{'criterion': 'gini', 'max_depth': 15, 'min_sa...",0.7740,0.7725,0.7830,0.7805,0.7725,0.7765,0.004393,14
2,0.016178,0.000767,0.001750,0.000040,gini,20,124,88,"{'criterion': 'gini', 'max_depth': 20, 'min_sa...",0.7705,0.7745,0.7990,0.7790,0.7735,0.7793,0.010221,8
3,0.021549,0.001860,0.002226,0.000782,gini,None,46,62,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.7575,0.7730,0.7725,0.7835,0.7560,0.7685,0.010378,16
4,0.016030,0.000361,0.001855,0.000087,gini,15,126,84,"{'criterion': 'gini', 'max_depth': 15, 'min_sa...",0.7705,0.7735,0.7995,0.7790,0.7745,0.7794,0.010413,7


Em seguida, imprimiremos os resultados e o melhor conjunto na tela:

In [41]:
scores = cross_val_score(busca, 
                         x_azar, y_azar, 
                         cv = KFold(n_splits=5, shuffle=True))

imprime_scores(scores)
melhor = busca.best_estimator_
print(melhor)

Accuracy médio: 78.71%
Intervalo: 77.49% a 79.93%
DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=71,
                       min_samples_split=100)


Nossa acurácia foi bem próxima dos resultados anteriores, mas o ponto é que demoramos um tempo 8.000 vezes menor para explorar esse espaço de parâmetros, obtendo resultados tão bons quanto conseguiríamos com o GridSearchCV.

## Será que vale a pena explorar mais?

Queremos ordenar os resultados da nossa busca pelo score médio (mean_test_score). Nesse momento, não estamos levando em consideração o intervalo de confiança (com duas vezes o desvio padrão).

In [42]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

Com a função iterrows, iremos iterar por cada uma das linhas dessa tabela do pandas. O iterrows é um gerador de iteração que devolve dois elementos em cada uma das linhas: o índice e a linha. Começaremos imprimindo os índices:

In [43]:
for indice, linha in resultados_ordenados_pela_media.iterrows():
  print(indice)

0
6
12
8
13
11
4
2
7
15
14
10
9
1
5
3


Esses são os índices ordenados do maior mean_test_score para o menor.

Agora, imprimiremos o mean_test_score, o desvio padrão do teste (std_test_score) e os parâmetros que geraram esse resultado (params, que devolve um objeto com todos os valores parametrizados).

Multiplicando o std_test_score por 2, chegaremos a um intervalo aproximado do que seria o desvio padrão. Por fim, definiremos que mean_test_score e std_test_score terão três casas decimais de ponto flutuante:

In [44]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media.iterrows():
  print(f'{linha.mean_test_score:.3f} +/-',
        f'{linha.std_test_score * 2:.3f}; ',
        f'{linha.params}')

0.787 +/- 0.019;  {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 71, 'min_samples_split': 100}
0.784 +/- 0.024;  {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 73, 'min_samples_split': 72}
0.784 +/- 0.024;  {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 67}
0.781 +/- 0.017;  {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 108, 'min_samples_split': 110}
0.780 +/- 0.019;  {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 125, 'min_samples_split': 59}
0.780 +/- 0.012;  {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 103, 'min_samples_split': 96}
0.779 +/- 0.021;  {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 126, 'min_samples_split': 84}
0.779 +/- 0.020;  {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 124, 'min_samples_split': 88}
0.779 +/- 0.009;  {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 101, 'min_samples_split': 52}
0.779 +/- 0.014;  {'criterion': '

Essa é uma forma resumida de imprimir os resultados que recebíamos na tabela do pandas.

Com 16 combinações, é uma análise razoável. Mas e se quiséssemos explorar um número maior - por exemplo, 64? Imprimindo os resultados dessa exploração na tela, encontraremos os mesmos RandomizedSearchCV irá explorar os parâmetros da mesma maneira.

Para encerrar, faremos a validação cruzada aninhada e imprimiremos o melhor conjunto de parâmetros encontrado para esse estimador:

In [45]:
scores = cross_val_score(busca,
                         x_azar, y_azar,
                         cv = KFold(n_splits=5, shuffle=True))

imprime_scores(scores)
melhor = busca.best_estimator_
print(melhor)

Accuracy médio: 78.70%
Intervalo: 76.85% a 80.55%
DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=71,
                       min_samples_split=100)


Esse é o resultado do nosso treino com uma busca aleatória contendo 64 tentativas. Repare que ainda conseguimos executar o código rapidamente, e com um computador mais potente conseguiríamos rodar ainda mais valores para o nosso estimador.

Mas será que o RandomizedSearchCV é mesmo melhor que o GridSearchCV?

# Testando um GridSearch mais longo

É hora de compararmos os resultados do GridSearchCV com os do RandomizedSearchCV.

Logicamente, estamos utilizando um exemplo de cada um desses algorítimos. É possível encontrar, na literatura e na prática, outros exemplos mostrando que buscar por completo um espaço discretizado com GridSearchCV trará a certeza de que os valores encontrados são os mais otimizados dentro desse espaço. 

Porém, o RandomizedSearchCV permite um controle maior sobre o tempo e o custo computacional/financeiro de otimização do modelo.

Além disso, se o grid tiver valores infinitos entre 0 e 1, será impossível explorar todo esse espaço, sendo necessário pegar exemplares aleatórios ou discretizar a seleção de alguma forma.

Começaremos nossa comparação pegando o código que criamos para GridSearchCV

In [46]:
SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    'max_depth' : [3, 5],
    'min_samples_split' : [32, 64, 128],
    'min_samples_leaf' : [32, 64, 128],
    'criterion' : ['gini', 'entropy'] 
}

busca = GridSearchCV(DecisionTreeClassifier(),
                     espaco_de_parametros,
                     cv = KFold(n_splits=5, shuffle=True))

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
# resultados.head()

Até o momento, vínhamos utilizando o DecisionTreeClassifier, um dos diversos classificadores baseados em árvores de decisão. Existem outros classificadores que, ao invés de tentarem uma única árvore, tentam diversas árvores. Um desses, bem famoso, é o ensemble RandomForestClassifier.

O sklearn ensemble RandomForestClassifier é um conjunto de classificadores que atuam de forma uníssona para chegar a uma conclusão. Além de possuir os diversos hiperparâmetros que já conhecemos antes, esse classificador possui alguns novos, como max_features (o número máximo de colunas de X utilizado para chegar a uma decisão), e o n_estimators (a quantidade de estimadores que serão treinados), para o qual atribuiremos os valores 10 e 100.

Com essas atribuições, já temos 72 combinações a serem exploradas. Porém, usaremos mais um último parâmetro, chamado BOOTSTRAP.

Ao invés do algorítimo tentar treinar os classificadores para todos os dados que estamos passando, correndo o risco de um overfitting, cada árvore é treinada com uma amostra desses dados. O bootstrap permite definir se um mesmo elemento pode fazer parte de diferentes amostras. Passando os valores True e False, dobraremos o nosso espaço de parâmetros, terminando com 144 combinações.

Antes de rodarmos a busca, não iremos medir somente a acurácia, mas também o tempo gasto computacionalmente para chegarmos aos nossos modelos. Para isso usaremos o %%time

In [47]:
%%time

from sklearn.ensemble import RandomForestClassifier

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    'n_estimators' : [10, 100],  #aqui
    'max_depth' : [3, 5],
    'min_samples_split' : [32, 64, 128],
    'min_samples_leaf' : [32, 64, 128],
    'bootstrap' : [True, False],  #aqui
    'criterion' : ['gini', 'entropy'] 
}

busca = GridSearchCV(RandomForestClassifier(),
                     espaco_de_parametros,
                     cv = KFold(n_splits=5, shuffle=True))

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
# resultados.head()

CPU times: user 3min 32s, sys: 533 ms, total: 3min 33s
Wall time: 3min 34s


Na tela serão impressos o dataframe com os nossos resultados e o tempo total dessa execução - no nosso caso, 4'8'' minutos. 

Vamos imprimir os 5 melhores resultados:

In [48]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  print(f'{linha.mean_test_score:.3f} +/- ',
        f'{linha.std_test_score * 2:.3f} ;',
        f'{linha.params}')

0.780 +/-  0.020 ; {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 64, 'n_estimators': 10}
0.778 +/-  0.020 ; {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 128, 'n_estimators': 10}
0.778 +/-  0.030 ; {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 10}
0.778 +/-  0.027 ; {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 10}
0.778 +/-  0.033 ; {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 64, 'n_estimators': 100}


Conseguimos uma média de 0.78 e um desvio padrão bem controlado, de apenas 0.02. Agora rodaremos o código do cross_validation_score, também medindo o tempo dessa execução:

In [49]:
# %%time

# scores = cross_val_score(busca,
#                          x_azar, y_azar,
#                          cv = KFold(n_splits=5, shuffle=True))

# imprime_scores(scores)
# melhor = busca.best_estimator_
# print(melhor)

Esse processo irá demorar tanto que o próprio Google Colab encerrará a conexão com a máquina virtual do Python. Ou seja, seria necessário rodarmos o código na nossa própria máquina para que a execução chegasse ao seu fim.

## Comparando com o RandomizedSearch

Para começarmos a comparação com o RandomizedSearchCV, copiaremos o código criado na aula anterior.

Substituiremos o campo GridSearchCV() por RandomizedSearchCV(), mantendo exatamente os mesmos parâmetros, com a exceção de n_iter = 20 - ou seja, buscaremos 20 iterações nesse espaço de parâmetros.

In [50]:
%%time

from sklearn.ensemble import RandomForestClassifier

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    'n_estimators' : [10, 100],  
    'max_depth' : [3, 5],
    'min_samples_split' : [32, 64, 128],
    'min_samples_leaf' : [32, 64, 128],
    'bootstrap' : [True, False],  
    'criterion' : ['gini', 'entropy'] 
}

busca = RandomizedSearchCV(RandomForestClassifier(),  #aqui
                     espaco_de_parametros,
                     n_iter= 20,  #aqui
                     cv = KFold(n_splits=5, shuffle=True))

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
# resultados.head()

CPU times: user 31.6 s, sys: 80.8 ms, total: 31.6 s
Wall time: 31.9 s


No nosso caso, essa execução levou cerca de 34.5 segundos. Vamos imprimir os 5 melhores resultados:

In [51]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  print(f'{linha.mean_test_score:.3f} +/-',
        f'{linha.std_test_score * 2:.3f} ;',
        f'{linha.params}')

0.776 +/- 0.025 ; {'n_estimators': 100, 'min_samples_split': 32, 'min_samples_leaf': 32, 'max_depth': 5, 'criterion': 'gini', 'bootstrap': False}
0.776 +/- 0.023 ; {'n_estimators': 100, 'min_samples_split': 32, 'min_samples_leaf': 128, 'max_depth': 3, 'criterion': 'gini', 'bootstrap': False}
0.776 +/- 0.024 ; {'n_estimators': 100, 'min_samples_split': 64, 'min_samples_leaf': 32, 'max_depth': 5, 'criterion': 'entropy', 'bootstrap': True}
0.776 +/- 0.032 ; {'n_estimators': 10, 'min_samples_split': 32, 'min_samples_leaf': 64, 'max_depth': 3, 'criterion': 'entropy', 'bootstrap': False}
0.775 +/- 0.035 ; {'n_estimators': 10, 'min_samples_split': 32, 'min_samples_leaf': 32, 'max_depth': 5, 'criterion': 'gini', 'bootstrap': True}


Quando exploramos as 144 combinações do nosso grid, tínhamos chegado à média 0.780 com +- 0.020 de desvio padrão - ou seja, valores muito próximos dos que encontramos com o RandomSearchCV. Lembrando que esses valores são relativamente próximos - ou seja, essa interpretação depende muito da situação em que nosso algorítimo é aplicado. Em casos de vida ou morte, por exemplo, uma diferença de 0.004 pode ser significante.

Dessa vez, com relação ao tempo, é até viável executarmos a exploração do cross_validation_score():

In [52]:
%%time

scores = cross_val_score(busca,
                         x_azar, y_azar,
                         cv = KFold(n_splits=5, shuffle=True))

imprime_scores(scores)
melhor = busca.best_estimator_
print(melhor)

Accuracy médio: 77.59%
Intervalo: 76.47% a 78.71%
RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_leaf=32,
                       min_samples_split=32)
CPU times: user 2min 11s, sys: 370 ms, total: 2min 11s
Wall time: 2min 11s


Em cerca de 2 minutos e meio obtivemos os resultados do cross_validation_score() com o RandomizedSearchCV.

Enquanto isso, somente com 144 possibilidades, não conseguimos rodar a mesma função com do GridSearchCV remotamente.

Imagine então se, para min_samples_split e min_samples_leaf, utilizássemos o parâmetro randint para iterar entre qualquer número entre 32 e 129? Ou mesmo para iterar entre 10 e 101 em n_estimators e entre 3 e 6 em max_depth?

Nesse caso, teríamos 10.274.628 combinações (91*3*97*97*2*2). Parece inviável, não é? Já com o RandomSearchCV, poderíamos até mesmo controlar o tempo (e o custo computacional) dispensado à essa tarefa. 

Por exemplo, se levamos cerca de meio minuto para iterar por 20 possibilidades randômicas (n_iter= 20), podemos estimar que iterar por 80 possibilidades levará cerca de 2 minutos. Vamos testar?

In [53]:
%%time

from sklearn.ensemble import RandomForestClassifier

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    'n_estimators' : randint(10, 101),  #aqui e demais abaixo
    'max_depth' : randint(3, 6),
    'min_samples_split' : randint(32, 129),
    'min_samples_leaf' : randint(32, 129),
    'bootstrap' : [True, False],  
    'criterion' : ['gini', 'entropy'] 
}

busca = RandomizedSearchCV(RandomForestClassifier(),  
                     espaco_de_parametros,
                     n_iter= 80,  #aqui
                     cv = KFold(n_splits=5, shuffle=True))

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
# resultados.head()

CPU times: user 1min 46s, sys: 260 ms, total: 1min 46s
Wall time: 1min 46s


Nesse caso, levamos cerca de 125 segundos (2 minutos) para rodar o código - ou seja, nossa estimativa deu certo. Lembre-se que esse tipo de cálculo vai depender do algorítimo e de suas especificidades.

Dentro desse espaço de parâmetros, vamos imprimir as 5 melhores combinações:

In [54]:
resultados_ordenados_pela_media = resultados.sort_values('mean_test_score', ascending = False)

for indice, linha in resultados_ordenados_pela_media[:5].iterrows():
  print(f'{linha.mean_test_score:.3f} +/-',
        f'{linha.std_test_score * 2:.3f} ;',
        f'{linha.params}')

0.779 +/- 0.025 ; {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 84, 'min_samples_split': 89, 'n_estimators': 48}
0.778 +/- 0.031 ; {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 32, 'min_samples_split': 96, 'n_estimators': 18}
0.778 +/- 0.032 ; {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 121, 'min_samples_split': 47, 'n_estimators': 27}
0.778 +/- 0.024 ; {'bootstrap': False, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 96, 'min_samples_split': 98, 'n_estimators': 11}
0.777 +/- 0.029 ; {'bootstrap': True, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 63, 'min_samples_split': 88, 'n_estimators': 69}


O melhor resultado que encontramos foi 0.79 de média com +- 0.025 de desvio padrão, muito próximo dos anteriores. Lembrando que, com o GridSearchCV, levamos cerca de 4 minutos e meio para chegar aos resultados explorando um espaço muito menor.

Repare que, mesmo nesse espaço enorme de mais de 10 milhões de combinações, não tivemos uma variabilidade muito grande de resultados. Mesmo os últimos 5 elementos dessa lista, que têm uma qualidade menor, não são tão discrepantes.

Dependendo do algorítimo e dos dados, pode ser que a escolha de um hiperparâmetro faça uma diferença muito grande no sistema como um todo. Como exemplo, você pode consultar o artigo Hyperparameters Matter, que analisa a importância dos hiperparâmetros no contexto de recomendações com Word2vec.

Ainda falta explorarmos um espaço que não seja baseado em árvores de decisão, como o SVC. A seguir, iremos estudar como explorar dois tipos de algorítimos ao mesmo tempo dentro do SVC.



# Treino teste validação, otimização sem validação cruzada

Agora que fizemos algumas comparações entre o GridSearchCV e o RandomizedSearchCV, vamos analisar alguns casos diferentes.

Por exemplo, pode ser que não seja possível, computacionalmente, rodar um cross validation, independentemente do fold. Nesse caso, como faríamos uma otimização de hiperparâmetros sem cross validation? Teríamos que, mesmo assim, tentar separar os dados entre treino e teste.

Até o momento, estávamos trabalhando com duas fases: a fase de treino e teste, e a fase de validação com cross_val_score() (nested cross validation).

Na prática, agora teremos três fases: uma fase de treino do modelo (ou de vários modelos) na busca de otimizar os hiperparâmetros; uma fase de teste, comparando os modelos para encontrar os melhores resultados; e uma fase de validação, tentando alcançar uma estimativa real desse algorítimo.

Ou seja, teremos que separar três conjuntos de dados, e não mais dois, como vínhamos fazendo com a função train_test_split().

No sklearn.model_selection, precisaremos encontrar um algorítimo de separação que não seja um KFold (que só separa uma única vez, sem validação cruzada). Existem algorítimos que fazem isso, como o ShuffleSplit, que irá aleatorizar os dados e quebrá-los uma única vez; ou o StratifiedShuffleSplit, que irá aleatorizar a ordem dos dados e quebrá-los de acordo com a estratificação dos dados que passarmos para ele. É esse algorítimo que utilizaremos agora, independentemente de trabalharmos com o GridSearchCV ou com o RandomizedSearchCV.

Para começar, copiaremos o último código que escrevemos para RandomizedSearchCV. Nele, faremos a importação do StratifiedShuffleSplit e criaremos uma variável split recebendo a parametrização desse algorítimo - no nosso caso, n_splits = 1 e test_size = 0.2 (reservando apenas 20% dos nossos dados para o teste).

Ao invés de 80 iterações, faremos apenas 5, acelerando a execução do código:

In [55]:
%%time

from sklearn.model_selection import StratifiedShuffleSplit

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    'n_estimators' : randint(10, 101),
    'max_depth' : randint(3, 6),
    'min_samples_split' : randint(32, 129),
    'min_samples_leaf' : randint(32, 129),
    'bootstrap' : [True, False],  
    'criterion' : ['gini', 'entropy'] 
}

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2)  #aqui

busca = RandomizedSearchCV(RandomForestClassifier(),  
                     espaco_de_parametros,
                     n_iter= 5,  #aqui
                     cv = split)  #aqui

busca.fit(x_azar, y_azar)
resultados = pd.DataFrame(busca.cv_results_)
# resultados.head()

CPU times: user 1.84 s, sys: 6 ms, total: 1.84 s
Wall time: 1.84 s


Quando fazemos um cross validation com 2 folds, ele executa duas vezes o algorítimo com 50% dos dados em cada uma das vezes.

Diferentemente disso, dessa vez separamos 80% dos dados para o teste, e 20% para o treino, rodando o algorítimo uma única vez.

Sem a cross validation, teremos que encontrar outra forma de obter os resultados finais desse algorítimo. Precisaremos, então, de um conjunto de dados inédito para executar a validação do nosso modelo. Mas como faremos isso se todos os dados foram utilizados no treino e no teste?

A resposta na verdade é bem simples: a separação desses dados deve ser feita de antemão.

Portanto, antes de treinarmos o modelo com x_azar e y_azar, separaremos uma amostra dos dados para a fase que chamaremos de validação.

Vamos supor que queremos 60% para treino, 20% para teste (também chamado de "dev teste") e 20% para a validação final. Faremos isso utilizando o train_test_split().

Para essa função, passaremos os dados x_zar, y_azar, e os parâmetros test_size:0,2, shuffle=True, stratify=y_azar. Nesse caso, estamos separando os 20% dos dados para validação, mesmo que o parâmetro do algorítimo se chame test_size.

Essa funçã nos devolve x_train, x_test, y_train e y_test. Vamos nomear cada um desses objetos como x_treino_teste, x_validacao, y_treino_teste, y_validacao.

Também precisaremos passar o SEED que nosso código seguirá. Para garantirmos que as dimensões dos dados estão separadas corretamente, imprimiremos todas aquelas variáveis na tela:

In [56]:
from sklearn.model_selection import train_test_split

SEED = 301
np.random.seed(SEED)

x_treino_teste, x_validacao, y_treino_teste, y_validacao = train_test_split(
    x_azar, y_azar, 
    test_size = 0.2,
    shuffle = True,
    stratify = y_azar
    )

print(x_treino_teste.shape)
print(x_validacao.shape)
print(y_treino_teste.shape)
print(y_validacao.shape)

(8000, 3)
(2000, 3)
(8000,)
(2000,)


Ou seja, temos:

* 8.000 elementos e 3 colunas para treino do algorítimo
* 2.000 elementos para teste
* 1 coluna para verificar as features e a classe do algorítimo

Agora, na função busca.fit(), deveremos passar as variáveis atualizadas (x_treino_teste e y_treino_teste). Também devemos nos atentar ao StratifiedShuffleSplit(): estamos passando test_size=0.2, mas 20% de 80% são 16%, ou seja, na verdade, precisamos atribuir test_size=0.25, 25%.

In [57]:
%%time

from sklearn.model_selection import StratifiedShuffleSplit

SEED = 301
np.random.seed(SEED)

espaco_de_parametros = {
    'n_estimators' : randint(10, 101),
    'max_depth' : randint(3, 6),
    'min_samples_split' : randint(32, 129),
    'min_samples_leaf' : randint(32, 129),
    'bootstrap' : [True, False],  
    'criterion' : ['gini', 'entropy'] 
}

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25)  #aqui

busca = RandomizedSearchCV(RandomForestClassifier(),  
                     espaco_de_parametros,
                     n_iter= 5,  
                     cv = split)  

busca.fit(x_treino_teste, y_treino_teste)  #aqui
resultados = pd.DataFrame(busca.cv_results_)
# resultados.head()

CPU times: user 1.46 s, sys: 9.02 ms, total: 1.47 s
Wall time: 1.47 s


Agora podemos validar nossos estimadores com os dados que encontramos. A maneira mais simples de fazer isso é com o cross_val_score(), utilizando split ao invés de KFold. Além disso, passaremos x_validacao e y_validacao ao invés de x_azar e y_azar:

In [58]:
%%time

scores = cross_val_score(busca,
                         x_validacao, y_validacao,
                         cv = split)

scores

CPU times: user 572 ms, sys: 6.03 ms, total: 578 ms
Wall time: 577 ms


array([0.754])

O resultado é um único 0.754 - como só tivemos um teste e uma validação, removemos a impressão da média e do intervalo.

O cross validation é um processo bastante interessante e prático, e inclusive poderíamos criar um pipeline que o fizesse de uma só vez. Porém, quando existem motivos para não utilizarmos o cross validation, devemos nos atentar a alguns detalhes importantes - por exemplo, à perda do intervalo de resultados.

Nós ainda poderíamos rodar o algorítimo StratifiedShuffleSplit() mais de uma vez (n_splits=5, por exemplo), obtendo resultados mais parecidos com um processo de cross validation - inclusive com diversos scores para analisarmos. Porém, as proporções podem ser diferentes, o que exigiria alguns cuidados.

# Skopt e o Hyperopt

Também existem outras bibliotecas para Python, como o skopt e o hyperopt, que vão tentar fazer uma busca aleatória de maneira mais inteligente - lembrando que esse tipo de busca não garante mínimos ou máximos globais.